In [6]:
# Install required libraries
!pip install -q yfinance pandas gradio scikit-learn matplotlib tensorflow pillow

In [7]:
# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
import io
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [8]:
stock_options = {
    'Apple (AAPL) 📱': 'AAPL',
    'Google (GOOGL) 🌐': 'GOOGL',
    'Microsoft (MSFT) 💻': 'MSFT',
    'Tesla (TSLA) 🚗': 'TSLA',
    'Amazon (AMZN) 📦': 'AMZN'
}


In [9]:
# Function to fetch, process, train, and predict
def get_stock_data(ticker):
    df = yf.download(ticker, period='5y')
    data = df[['Close']].dropna()

    # Normalize
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences
    X, y = [], []
    for i in range(60, len(scaled_data)):
        X.append(scaled_data[i-60:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=5, batch_size=32, verbose=0)

    # Predict next closing price
    last_60 = scaled_data[-60:]
    last_60 = np.reshape(last_60, (1, 60, 1))
    prediction = model.predict(last_60)
    predicted_price = scaler.inverse_transform(prediction)[0][0]

    # Plot last 100 days with predicted price
    plt.figure(figsize=(8, 4))
    plt.plot(data.index[-100:], data['Close'][-100:], label="Actual Price")
    plt.axhline(y=predicted_price, color='r', linestyle='--', label="Predicted Next Price")
    plt.title(f"{ticker} - Last 100 Days & Next Prediction")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.tight_layout()

    # Convert plot to image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)
    plt.close()

    return f"Predicted next closing price: ${predicted_price:.2f} 💰", img

In [10]:

# Gradio app
def app(label):
    ticker = stock_options[label]
    return get_stock_data(ticker)

gr.Interface(
    fn=app,
    inputs=gr.Dropdown(choices=list(stock_options.keys()), label="Select a Stock 📊"),
    outputs=["text", "image"],
    title="Stock Price Prediction with LSTM 📈💹",
    description="Pick a stock to see its predicted next closing price using deep learning (LSTM model)."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1fc1fdee798fcbf749.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
